In [ ]:
import geopandas as gpd
import pandas as pd
import plotly.express as px

# Load election data
data_dir = "../data/"
income = pd.read_csv(data_dir + "income_by_municipality.csv", sep=',')
income

In [ ]:
# Convert the 'Total' column to numeric using .loc
income.loc[:, 'Total'] = income['Total'].apply(pd.to_numeric, errors='coerce')
income

In [ ]:
income_municipality = income.groupby('Municipalities')['Total'].mean().reset_index()
income_municipality

In [ ]:
# Load GeoJSON file
map_catalonia='municipality_map.geojson'
geo_data = gpd.read_file(data_dir + map_catalonia)

In [ ]:
# Translate column used to merge datasets and sort dataset based on this
geo_data.rename(columns={'nom_muni': 'Municipalities'}, inplace=True)
geo_data=geo_data.sort_values(by='Municipalities')

In [ ]:
# Merge municipalities map and income by municipality
merged_data = geo_data.merge(income_municipality, on='Municipalities')
merged_data = merged_data.rename(columns={'Total': 'Average income per household (€)'})
merged_data

In [ ]:
# Plot the average income on a map of Catalonian municipalities

fig = px.choropleth_mapbox(
    merged_data,
    geojson=merged_data.geometry,
    locations=merged_data.index,
    color='Average income per household (€)',  # Change to the column you want to visualize
    hover_name='Municipalities',
    mapbox_style="carto-positron",
    center={"lat": 41.8781, "lon": 1.7834},  # Center of Catalonia
    zoom=7,
    title='Average income by municipality'
)
fig.update_layout(height=600, width=800)
fig.update_geos(fitbounds="locations", visible=False)

fig.show()